# **Парсер для LMS Moodle**

# Базовые функции и библиотеки

In [1]:
import xml.etree.ElementTree as ET

In [2]:
#возврат путей xml файлов
def activities_directories ():
    file = "backup/moodle_backup.xml"
    direct = []
    directory = find_by_tag(file,"directory")
    modulename = find_by_tag(file,"modulename")
    for i in range( len(modulename) ):
        text_dir = "backup/" + directory[i] + "/" + modulename[i] + ".xml"
        direct.insert(i, text_dir)
    return direct

In [3]:
#возварт текста по тегу
def find_by_tag (file, tag):
    tree = ET.parse(file)
    root = tree.getroot()
    res = ""
    for text_in_root in root.iter(tag):
        res += str(text_in_root.text)
        res += "\n"
    listRes = list(res.split("\n"))
    listRes.pop( len(listRes)-1 )
    return listRes

In [4]:
#возварт текста по тегу из конкретного вложенного тега
def find_by_tag_from_tag (file, where, tag):
    tree = ET.parse(file)
    root = tree.getroot()
    res = ""
    for text_in_root in tree.findall(".//" + where + "/" + tag):
        res += str(text_in_root.text)
        res += "\n"
    listRes = list(res.split("\n"))
    listRes.pop( len(listRes)-1 )
    return listRes

In [5]:
#возврат всего файла
def read_all(file):
    tree = ET.parse(file)
    s = ET.dump(tree)
    return s

In [6]:
#удаление конкретных символов
def remove_symbols(symbols_to_remove, text):
    for symbol in symbols_to_remove:
        text = text.replace(symbol, "")
    return text

In [7]:
#удаление последнего символа
def remove_last_symbol(text):
    result_string = ""
    index = len(text)
    for i in range(index-1):
        result_string += text[i]
    return result_string

# Функции чтения отдельных XML файлов ресурсов

In [8]:
#чтение page
def page_read (file):
    strings_arr = find_by_tag(file, "content")
    return strings_arr

In [9]:
#чтение choise
def choiсe_read (file):
    text = find_by_tag(file, "text")
    count = find_by_tag(file, "maxanswers")
    return text

In [10]:
#чтение book
def book_read (file):
    all_content = []
    name = remove_symbols("'[]", str( find_by_tag(file, "name") ) )
    title = find_by_tag(file, "title")
    book_content = find_by_tag(file, "content")
    all_content.append(name)
    all_content.append(title)
    all_content.append(book_content)
    return all_content

In [11]:
#чтение feedback
def feedback_read (file):
    all_content = []
    new_parts = []
    questions = find_by_tag_from_tag(file, "item", "name")

    arr_content = find_by_tag(file, "presentation")
    str_content = ''.join(arr_content)
    parts = str_content.rsplit('>>>>>')
    parts.pop(0)

    #убираем последний символ
    for i in range ( len(parts) ):
        if i != len(parts)-1:
            new_parts.append( remove_last_symbol(parts[i]) )
        else:
            new_parts.append( parts[i] )
        
    all_content.append(questions)
    
    #разделяем строки по разделителям
    for i in range ( len(parts) ):
        new = new_parts[i].rsplit('|')
        all_content.append(new)
    
    return all_content

In [12]:
#чтение survey
def survey_read (file):
    all_content = []
    
    questions = find_by_tag(file, "questions")
    str_content = ''.join(questions)
    parts = str_content.rsplit(',')

    all_content.append(find_by_tag(file, "name"))
    all_content.append(parts)
    
    return all_content

# Функции чтения всех файлов

In [13]:
#чтаение всех видов файлов
def read_file (file):
    s = file
    parts = s.rsplit('/')
    res = parts[3]
    
    if res == "page.xml" :
        return page_read (file)
    elif res == "choice.xml" :
        return choiсe_read (file)
    elif res == "book.xml" :
        return book_read (file)
    elif res == "feedback.xml" :
        return feedback_read (file)
    elif res == "survey.xml" :
        return survey_read (file)

In [14]:
#чтение всего backup файла
def all_files_read ():
    all_content = []
    titles = find_by_tag_from_tag("backup/moodle_backup.xml", "activity", "title")
    types = find_by_tag("backup/moodle_backup.xml", "modulename")
    ids = find_by_tag("backup/moodle_backup.xml", "moduleid")
    urls = []
    contents = []
    dirs = activities_directories()
    
    for i in range ( len(dirs) ):
        contents.append( read_file(dirs[i]) )
        orig_url = str( find_by_tag("backup/moodle_backup.xml", "original_wwwroot") )

        #формирование ссылки
        symbols_to_remove = " '[] "
        orig_url = remove_symbols(symbols_to_remove, orig_url)
        urls.append( orig_url + "/mod/" + types[i] + "/view.php" +  "?id=" + ids[i])

    all_content.append(titles)
    all_content.append(types)
    all_content.append(urls)
    all_content.append(contents)
    return all_content

# Основной код

In [15]:
all_files_read()

[['Объявления',
  'Информация по экзамену',
  'Предсессионный опрос',
  'Запись на консультации',
  'Стандарты оформления в ИРНИТУ',
  'Опрос: начальные компетенции',
  'Анкета тест 1 COLLESф',
  'Введение в программирование',
  'Введение в программирование',
  'Отчёт по лабораторной',
  'Тест по основам программирования'],
 ['forum',
  'page',
  'choice',
  'scheduler',
  'resource',
  'feedback',
  'survey',
  'book',
  'lesson',
  'assign',
  'quiz'],
 ['http://51.250.4.123/moodle/mod/forum/view.php?id=1',
  'http://51.250.4.123/moodle/mod/page/view.php?id=3',
  'http://51.250.4.123/moodle/mod/choice/view.php?id=4',
  'http://51.250.4.123/moodle/mod/scheduler/view.php?id=5',
  'http://51.250.4.123/moodle/mod/resource/view.php?id=6',
  'http://51.250.4.123/moodle/mod/feedback/view.php?id=7',
  'http://51.250.4.123/moodle/mod/survey/view.php?id=8',
  'http://51.250.4.123/moodle/mod/book/view.php?id=9',
  'http://51.250.4.123/moodle/mod/lesson/view.php?id=10',
  'http://51.250.4.123/mo